# SonoPleth - Audio Metadata Processing Pipeline

This notebook runs the complete pipeline:
1. **Check audio channels** - Scan which channels contain audio
2. **Extract metadata** - Run bwfmetaedit to get ADM XML
3. **Parse metadata** - Extract spatial objects and DirectSpeakers
4. **Create spatial instructions** - Generate CSV with channel positions and timing

**Outputs:**
- `processedData/containsAudio.json` - Channel audio presence
- `processedData/currentMetaData.xml` - Extracted ADM metadata
- `processedData/objectData.json` - Parsed audio objects
- `processedData/directSpeakerData.json` - Parsed DirectSpeaker channels
- `processedData/globalData.json` - Global technical metadata
- `forExport/spatialInstructions.csv` - Final spatial instructions

## Setup: Import modules and set source file

In [ ]:
from src.extractMetadata import extractMetaData
from src.parser import parseMetadata
from src.createSpatialInstructions import createSpatialInstructionsCSV, loadProcessedData
from src.checkAudioChannels import exportAudioActivity
import json

sourceADMFile = "sourceData/POE-ATMOS-FINAL.wav"
print(f"Source file: {sourceADMFile}")

## Step 1: Check Audio Channels

Scans all channels in the WAV file to detect which contain audio above a threshold (-100 dBFS by default).
This helps filter empty/silent channels before processing.

In [ ]:
print("Checking audio channels for content...\n")
exportAudioActivity(sourceADMFile, output_path="processedData/containsAudio.json", threshold_db=-100)

# Show summary
with open("processedData/containsAudio.json", "r") as f:
    audio_data = json.load(f)
    
active_channels = sum(1 for ch in audio_data["channels"] if ch["contains_audio"])
total_channels = len(audio_data["channels"])
print(f"\n✓ Found {active_channels}/{total_channels} channels with audio")

## Step 2: Extract ADM Metadata

Uses `bwfmetaedit` to extract ADM (Audio Definition Model) metadata from the WAV file's aXML chunk.
This contains spatial positioning data for all audio objects and DirectSpeakers.

In [ ]:
print("Extracting ADM metadata from WAV file...\n")
extractedMetadata = extractMetaData(sourceADMFile, "processedData/currentMetaData.xml")

if extractedMetadata:
    xmlPath = extractedMetadata
    print(f"\n✓ Metadata extracted to: {xmlPath}")
else:
    xmlPath = "data/POE-ATMOS-FINAL-metadata.xml"
    print(f"\n⚠ Using fallback XML: {xmlPath}")

## Step 3: Parse ADM Metadata

Parses the ADM XML to extract:
- **Audio objects** - Dynamic sources with position/time data
- **DirectSpeakers** - Static speaker positions (7.1.4 bed)
- **Global metadata** - File info (sample rate, duration, etc.)

In [ ]:
print("Parsing ADM metadata...\n")
reformattedMetadata = parseMetadata(xmlPath, ToggleExportJSON=True, TogglePrintSummary=True)

# Load and display summary
data = loadProcessedData("processedData")

print(f"\n✓ Parsed metadata:")
print(f"  DirectSpeakers: {len(data.get('directSpeakerData', {}))}")
print(f"  Audio objects: {len([k for k,v in data.get('objectData', {}).items() if v])}")
print(f"  Sample rate: {data.get('globalData', {}).get('SampleRate', 'N/A')} Hz")
print(f"  Duration: {data.get('globalData', {}).get('Duration', 'N/A')}")

## Step 4: Create Spatial Instructions CSV

Generates a CSV with timestamped spatial data for all channels:
- **Static channels first** (DirectSpeakers + non-moving objects)
- **Dynamic channels second** (objects with position changes)

Each row includes: `channel`, `id`, `has_audio`, `time`, `x`, `y`, `z`, `dynamic`

In [ ]:
print("Creating spatial instructions CSV...\n")
row_count = createSpatialInstructionsCSV(
    processed_dir="processedData",
    output_path="forExport/spatialInstructions.csv"
)

print(f"\n✓ CSV created with {row_count} rows")
print(f"  Output: forExport/spatialInstructions.csv")

## Summary

Pipeline complete! Files created:
- `processedData/containsAudio.json`
- `processedData/currentMetaData.xml`
- `processedData/objectData.json`
- `processedData/directSpeakerData.json`
- `processedData/globalData.json`
- `forExport/spatialInstructions.csv`

## Optional: View CSV Preview

In [ ]:
import pandas as pd

# Load and display first 20 rows of the CSV
df = pd.read_csv("forExport/spatialInstructions.csv")
print(f"CSV shape: {df.shape[0]} rows × {df.shape[1]} columns\n")
print("First 20 rows:")
df.head(20)